# Estimating effective reproduction number $R_{t}$ for different provinces across Canada
> Bayesian modelling inspired by Bettencourt & Ribeiro's Approach to estimate daily values of $R_{t}$

- toc: true
- author: Ujala S
- categories: [reproduction number, COVID, Canada]
- image: images/ujs_rt_canada.png
- permalink: /rt-canada/

In [9]:
#hide
!pip install altair

In [10]:
#hide
import pandas as pd
import numpy as np
import altair as alt
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
import seaborn as sns
import math
from scipy import stats as sps
from IPython.display import clear_output

## Introduction
Effective reproduction number ($R_{t}$) has been used as one of the primary metrics to assess the state of an ongoing epidemic or pandemic. It is the number of secondary infections caused by a primary case at time t. In other words, the number of people who get infected per infectious person. This metric is very informative- if $R_{t}$ is greater than one, that indicates that the pandemic is spreading fast, if it is less than 1 that means the pandemic is slowing down. So the end goal should be to reduce $R_{t}$ to manage a pandemic. Daily $R_{t}$ values can be used in evaluating the impact of policy measures (for example- wearing masks, economic lockdown, social distancing etc) on controlling the  pandemic. $R_{t}$ values can vary with time and location based on changing restrictions and behaviours. 
Hence, I wanted to look at the situation of my current country of residence, Canada. I used Bayesian modelling to estimate daily $R_{t}$ values of larger provinces in Canada with a population of over a million.

In [11]:
#hide
#Load and preprocess data

def get_process_data(url = 'https://health-infobase.canada.ca/src/data/covidLive/covid19-download.csv', 
                     regions = ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec']):
  """Loads data from source to dataframe & preprocesses it"""
  # Select provinces with population > 1 million
  data = pd.read_csv(url,
                     usecols=['date', 'prname', 'numtotal'],
                     parse_dates=['date'],
                     index_col=['prname', 'date'],
                     squeeze=True).sort_index()



  idx = pd.IndexSlice
  data = data.loc[idx[regions,:]]
  return data


In [12]:
#hide
data = get_process_data()

## Bayesian Modelling Approach
The main steps in the modelling process are:
*   Loading data from the source and preprocessing it. The main variable we are interested in is the total number of covid cases on a given date.
*   Calculate the new cases each day and apply some Gaussian noise and moving average computation to smooth the data. I do this to shift the data closer to the truth and adjust for errors in reporting etc. This is not a perfect approach but gives reasonable results. Smoothed data is monotonically increasing and a cutoff is applied to start the analysis from a date when the new cases become consistent enough to consider an epidemic breakout.
*   $R_{t}$ is treated like a random variable with posterior probability distribution of $P(R_{t}|k)$ where k is number of new cases each day. In order to compute posterior probabilities, we need to use the [Bayes Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem). The main elements of the computation are as follows:
  *  $P(R_{t})$- Prior distribution of $Rt$. According to Bettencourt & Ribeiro's approach, the distribution of $R_t$ is assumed to be a Gaussian centered around $R_{t-1}$, so $P(R_{t}|R_{t-1})=\mathcal{N}(R_{t-1}, \sigma)$, where $\sigma$ is a hyperparameter. More details on the model can be found [here](https://github.com/k-sys/covid-19/blob/master/Realtime%20R0.ipynb).
  *  $P(k|R_{t})$- The likelihood of k new cases given $R_{t}$. We can look at 'k' new cases as arrivals and model it using the [Poisson Distribution](https://en.wikipedia.org/wiki/Poisson_distribution) with an arrival rate of lambda ($\lambda$).
  *  $P(k)$- The unconditional probability of seeing k new cases in general. This is computed by applying the [total probability theorem](https://en.wikipedia.org/wiki/Law_of_total_probability) over the numerator of the Bayes formula.
*   We use maximum likelihhod estimation to choose the value of $\sigma$ that maximizes the likelihood of the data $P(k)$.
*   Lastly, we compute the [highest density intervals](https://www.sciencedirect.com/topics/mathematics/highest-density-interval) for $R_t$ and plot daily $R_t$ on time series chart.






In [13]:
#hide
#Smoothing the time series data (New Daily Cases) by applying Gaussian Filter

def smooth_cases(cases, cutoff = 10):
  new_cases = cases.diff()

  smoothed = new_cases.rolling(7, win_type = 'gaussian', min_periods = 1, center = True).mean(std = 2).round()
  idx_start = np.searchsorted(smoothed, cutoff)
  smoothed = smoothed.iloc[idx_start:]
  original = new_cases.loc[smoothed.index]

  return original, smoothed

In [14]:
#hide
#Calculating posterior probabilities for Reproduction Number
def get_posteriors(new_cases_smoothed, sigma = 0.15, gam = 1/7, r_t_max = 12):
  '''Function to compute posterior probabilities. Input is daily new cases (smoothed), 
  standard deviation for prior distribution, 
  gamma value(parameter used to compute lambda), and maximum value for Rt'''

  r_t_range = np.linspace(0, r_t_max, r_t_max*100+1)
  # Calculate Lambda
  lam = new_cases_smoothed[:-1].values * np.exp(gam * (r_t_range[:, None] - 1)) 
  

  #Calculate P(k|Rt)
  likelihoods = pd.DataFrame(data = sps.poisson.pmf(new_cases_smoothed[1:], lam),
                             index = r_t_range,
                             columns = new_cases_smoothed.index[1:])
 
  
  # Create Gaussian and normalize. Used to calculate priors
  process_matrix = sps.norm(loc=r_t_range,
                              scale=sigma
                             ).pdf(r_t_range[:, None]) 

  process_matrix /= process_matrix.sum(axis=0)

  # Calculate the initial prior
  prior0 = np.ones_like(r_t_range)/len(r_t_range)
  prior0 /= prior0.sum()

  # Create a DataFrame that will hold posteriors for each day
    # Insert prior as the first posterior.
  posteriors = pd.DataFrame(
      index=r_t_range,
      columns=new_cases_smoothed.index,
      data={new_cases_smoothed.index[0]: prior0}   
    )   

  # Keeping track of data for maximum likelihood calculation.
  log_likelihood = 0.0

    # Iteratively apply Bayes' rule
  for previous_day, current_day in zip(new_cases_smoothed.index[:-1], new_cases_smoothed.index[1:]):

        # Calculate the new prior
      current_prior = process_matrix @ posteriors[previous_day]
        
        # Calculate the numerator of Bayes' Rule: P(k|R_t)P(R_t)
      numerator = likelihoods[current_day] * current_prior
        
        # Calcluate the denominator of Bayes' Rule P(k)
      denominator = np.sum(numerator)
        
        # Execute full Bayes' Rule
      posteriors[current_day] = numerator/denominator
        
        # Add to the running sum of log likelihoods
      log_likelihood += np.log(denominator)
    
  return posteriors, log_likelihood




In [15]:
#hide
def highest_density_interval(pmf, p=.9, debug=False):
    # If we pass a DataFrame, just call this recursively on the columns
    if(isinstance(pmf, pd.DataFrame)):
        return pd.DataFrame([highest_density_interval(pmf[col], p=p) for col in pmf],
                            index=pmf.columns)
    
    cumsum = np.cumsum(pmf.values)
    
    # N x N matrix of total probability mass for each low, high
    total_p = cumsum - cumsum[:, None]
    
    # Return all indices with total_p > p
    lows, highs = (total_p > p).nonzero()
    
    # Find the smallest range (highest density)
    best = (highs - lows).argmin()
    
    low = pmf.index[lows[best]]
    high = pmf.index[highs[best]]
    
    return pd.Series([low, high],
                     index=[f'Low_{p*100:.0f}',
                            f'High_{p*100:.0f}'])




In [16]:
#hide
def calc_results(data):

  sigmas = np.linspace(1/20, 1, 20)
  results = {}
  for region_name, cases in data.groupby(level='prname'):
    print('Calculating, this will take a few minutes- thanks for waiting :-)')
    print(region_name)
    new, smoothed = smooth_cases(cases, cutoff = 10)
    
    result = {}
    
    # Holds all posteriors with every given value of sigma
    result['posteriors'] = []
    
    # Holds the log likelihood across all k for each value of sigma
    result['log_likelihoods'] = []
    
    for sigma in sigmas:
      posteriors, log_likelihood = get_posteriors(smoothed, sigma=sigma)
      result['posteriors'].append(posteriors)
      result['log_likelihoods'].append(log_likelihood)
    
    # Store all results keyed off of state name
    results[region_name] = result
    clear_output(wait=True)

  # Each index of this array holds the total of the log likelihoods for
  # the corresponding index of the sigmas array.
  total_log_likelihoods = np.zeros_like(sigmas)

  # Loop through each state's results and add the log likelihoods to the running total.
  for state_name, result in results.items():
    total_log_likelihoods += result['log_likelihoods']

  # Select the index with the largest log likelihood total
  max_likelihood_index = total_log_likelihoods.argmax()

  # Select the value that has the highest log likelihood
  sigma = sigmas[max_likelihood_index]

  final_results = None

  for region_name, result in results.items():
    print('Calculating, this will take a few minutes- thanks for waiting :-)')
    print(region_name)
    posteriors = result['posteriors'][max_likelihood_index]
    hdis_90 = highest_density_interval(posteriors, p=.9)
    hdis_50 = highest_density_interval(posteriors, p=.5)
    most_likely = posteriors.idxmax().rename('Rt')
    result = pd.concat([most_likely, hdis_90, hdis_50], axis=1)
    if final_results is None:
        final_results = result
    else:
        final_results = pd.concat([final_results, result])
    clear_output(wait=True)


  return sigma, results, final_results

sigma, results, final_results = calc_results(data)

Calculating, this will take a few minutes- thanks for waiting :-)
Saskatchewan


## Results

### National

In [ ]:
#hide
data_national = get_process_data(regions = ['Canada'])
sigma_national, results_national, final_results_national = calc_results(data_national)




In [34]:
#hide_input
df_national = final_results_national.reset_index()
df_national = df_national[1:]
base = alt.Chart(df_national, title='Canada').mark_line(color = 'red').encode(x = alt.X('date'),
                                            y = alt.Y('Rt', title = 'Rt'),
                                            tooltip = ['date','Rt','Low_90','High_90']).interactive()


band = alt.Chart(df_national).mark_area(opacity=0.4, color='gray').encode(x='date',y= alt.Y('Low_90',title = ''),y2=alt.Y2('High_90',title=''))
base + band

alt.LayerChart(...)

### Provincial
A sanity check on the model can be done by comparing with latest $R_{t}$ forecasts [here](https://epiforecasts.io/covid/posts/national/canada/).

In [17]:
#hide_input
#Final Plotting
regions = ['Ontario','Quebec','British Columbia', 'Alberta','Saskatchewan','Manitoba']
df = final_results.reset_index()
for region in regions:
  df[df.prname == region] = df[df.prname == region][1:]
df.dropna(inplace = True)

base = alt.Chart(df).mark_line(color = 'red').encode(x = alt.X('date',title = ''),
                                            y = alt.Y('Rt', title = 'Rt'),
                                            tooltip = ['date','Rt','Low_90','High_90']).interactive()


band = alt.Chart(df).mark_area(opacity=0.4, color='gray').encode(x='date',y= alt.Y('Low_90', title=''),y2=alt.Y2('High_90',title=''))

alt.layer(base, band, data=df).facet('prname',columns = 3).resolve_axis(
    x='independent',
    y='independent',
)

alt.FacetChart(...)

## References
[1]:  Data sourced from ["Government of Canada"](https://www.canada.ca/en/public-health/services/diseases/2019-novel-coronavirus-infection.html).  
[2]:  Bayesian Model ["Bettencourt & Ribeiro's paper"](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0002185).  
[3]:  Project Inspiration ["Kevin Systrom's Blog"](http://systrom.com/blog/author/ksys1983/).  